In [1]:
!pip -q install gputil psutil humanize
# Import packages
import os,sys,humanize,psutil,GPUtil

# Define function
def mem_report():
  print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))

  GPUs = GPUtil.getGPUs()
  for i, gpu in enumerate(GPUs):
    print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'
    .format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))

# Execute function
mem_report()

  Preparing metadata (setup.py) ... done
CPU RAM Free: 12.4 GB
GPU 0 ... Mem Free: 15101MB / 15360MB | Utilization   0%


In [2]:
!nvidia-smi

Mon Dec 30 01:37:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Tesla T4 → Compute Capability 7.5 → Architecture 7.5

In [3]:
import os
os.environ['TORCH_CUDA_ARCH_LIST'] = "7.5"

In [4]:
!sudo apt-get install python3-pybind11

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libeigen3-dev pybind11-dev python3-numpy
Suggested packages:
  libeigen3-doc libmpfrc++-dev pybind11-doc python-numpy-doc python3-pytest
The following NEW packages will be installed:
  libeigen3-dev pybind11-dev python3-numpy python3-pybind11
0 upgraded, 4 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,826 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 pybind11-dev all 2.9.1-2 [146 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 python3-numpy amd64 1:1.21.5-1ubuntu22.04.1 [3,467 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-pybind11 all 2.9.1-2 [156 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libeigen3-dev all 3.4.0-2ubuntu2 [1,056 kB]
Fetched 4,826 kB in 1s 

In [8]:
!pip install ninja

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 8.1 MB/s eta 0:00:00


In [6]:
%%writefile matmul_kernel.cu
#include <torch/extension.h>

template <typename T>
__global__ void matmul_kernel(const T* A, const T* B, T* C, int M, int N, int K) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < M && col < N) {
        T sum = 0;
        for (int k = 0; k < K; ++k) {
            sum += A[row * K + k] * B[k * N + col];
        }
        C[row * N + col] = sum;
    }
}

template <typename T>
void matmul_launcher(torch::Tensor A, torch::Tensor B, torch::Tensor C) {
    int M = A.size(0); // Number of rows in A
    int K = A.size(1); // Number of columns in A
    int N = B.size(1); // Number of columns in B

    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks((N + 15) / 16, (M + 15) / 16);
    matmul_kernel<<<numBlocks, threadsPerBlock>>>(A.data_ptr<T>(), B.data_ptr<T>(), C.data_ptr<T>(), M, N, K);
}

torch::Tensor matmul_binding(torch::Tensor A, torch::Tensor B) {
    TORCH_CHECK(A.dim() == 2 && B.dim() == 2, "Input matrices must be 2-dimensional");
    TORCH_CHECK(A.size(1) == B.size(0), "Inner dimensions of matrices must match");

    auto C = torch::zeros({A.size(0), B.size(1)}, A.options()); // Create the result tensor on the same device
    AT_DISPATCH_FLOATING_TYPES(A.scalar_type(), "matmul_launcher", ([&] {
        matmul_launcher<scalar_t>(A, B, C);
    }));
    return C;
}

PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
    m.def("matmul", &matmul_binding, "Matrix multiplication kernel for dynamically sized matrices");
}


Overwriting matmul_kernel.cu


In [ ]:
from torch.utils.cpp_extension import load

matmul_kernel = load(
    name="matmul_kernel",
    sources=["matmul_kernel.cu"],
    extra_cuda_cflags=["-O3"]
)

In [10]:
import torch

def matmul_cuda(A, B):
    """
    Perform matrix multiplication on CUDA without Python-side dimension handling.
    A: (MxK) Tensor
    B: (KxN) Tensor
    Returns:
        C: (MxN) Tensor
    """
    return matmul_kernel.matmul(A, B)

# Example usage:
A = torch.randn(32, 64, device='cuda')  # MxK
B = torch.randn(64, 16, device='cuda')  # KxN
C = matmul_cuda(A, B)
print(C.shape)




torch.Size([32, 16])


In [11]:
import time
A = torch.randn(32, 64, device='cuda')  # MxK
B = torch.randn(64, 16, device='cuda')  # KxN

start = time.time()
C_pytorch = torch.matmul(A, B)
print("PyTorch Time:", time.time() - start)


start = time.time()
C = matmul_cuda(A, B)
print("CUDA Kernel Time:", time.time() - start)

PyTorch Time: 0.10501384735107422
CUDA Kernel Time: 0.00020503997802734375
